# Installing Dependencies

In [38]:
using DelimitedFiles
using ProgressMeter
using CUDA
using BenchmarkTools: @btime
include("../../../src/forces.jl")
include("../../../src/neighbor.jl")

cu_knn (generic function with 1 method)

# Initial Conditions

In [39]:
# Running fusion of the two aggregates
# Physical Conditions
R_agg = 10

# Time model Conditions
t_f = 10000
dt = 0.1

# Neighbor Conditions
n_knn = 50
nn = 12

# Forces Conditions
r_max = 2.8
fp = 0.002
K = 0.05
s = 2.0

# Coding C0nditions
n_text = 50
num_agg = 2

@time X = Float32.(readdlm("../../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

  0.001222 seconds (12.02 k allocations: 406.844 KiB)


746×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -2.89  -8.98
  0.5  -2.89  -8.98
  2.5  -2.89  -8.98
 -2.5  -1.15  -8.98
 -0.5  -1.15  -8.98
  1.5  -1.15  -8.98
  3.5  -1.15  -8.98
 -3.5   0.58  -8.98
 -1.5   0.58  -8.98
  0.5   0.58  -8.98
  2.5   0.58  -8.98
 -2.5   2.31  -8.98
 -0.5   2.31  -8.98
  ⋮           
  3.5  -2.31   8.98
 -3.5  -0.58   8.98
 -1.5  -0.58   8.98
  0.5  -0.58   8.98
  2.5  -0.58   8.98
 -2.5   1.15   8.98
 -0.5   1.15   8.98
  1.5   1.15   8.98
  3.5   1.15   8.98
 -1.5   2.89   8.98
  0.5   2.89   8.98
  2.5   2.89   8.98

# Using kNN

In [40]:
function diff_forces(n_knn, t_f, r_max, fp, K)
    # Definig Variables for calculing initial variables
    global X 
    X = X |> cu

    # Inizializate Variables for kNN
    global i_Cell = CuArray{Float32}(undef, (size(X, 1), size(X, 1), 3))
    global Dist = CuArray{Float32}(undef, (size(X, 1), size(X, 1)))
    global idx = hcat([[CartesianIndex(i,1) for i=1:nn] for j=1:size(X,1)]...) |> cu
    global rand_idx = CuArray{Int32}(undef, n_knn, size(X,1))

    # Inizializate Variables for Forces
    global r = zeros(nn,size(X)[1],3) |> cu
    global r_p = zeros(size(X)) |> cu
    global dist = zeros(nn, size(X)[1]) |> cu
    global dist_p = zeros(size(X,1)) |> cu
    global F = zeros(nn, size(X)[1],3) |> cu
    global dX = zeros(size(X)[1],3) |> cu;

    # Calculating position of every cell on the fusion
    p = Progress(Int(t_f/dt),barlen=25)
    for t in 0:Int(t_f/dt)
        ## LEAPFROG ALGORITHM
        if mod(t,2) == 0
            if mod(t, n_knn) == 0
                # Calculating kNN
                cu_knn()

                # # Calculating kNN
                # global idx = cpu_knn(Matrix(X), nn)

            end
            # Calculating Forces
            cu_forces(t, r_max, fp, K)
        else
            global X = X + dX*dt
        end

        next!(p)
    end
end

diff_forces (generic function with 3 methods)

In [41]:
diff_forces(n_knn, t_f, r_max, fp, K)

Progress: 100%|█████████████████████████| Time: 0:00:25


In [42]:
X

746×3 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
 -1.46872   -2.8616    -8.93854
  0.522198  -2.85615   -8.94166
  2.51764   -2.85286   -8.94185
 -2.46901   -1.13756   -8.93912
 -0.473787  -1.13034   -8.9436
  1.51894   -1.12663   -8.94321
  3.51379   -1.12623   -8.93554
 -3.4725     0.585913  -8.93843
 -1.47453    0.59369   -8.94169
  0.519949   0.598723  -8.94104
  2.51054    0.600824  -8.93729
 -2.47782    2.31484   -8.93554
 -0.486591   2.32384   -8.93725
  ⋮                    
  3.49567   -2.33436    8.92543
 -3.4791    -0.594511   8.93055
 -1.48096   -0.596763   8.93762
  0.515843  -0.598753   8.93807
  2.50758   -0.602109   8.93586
 -2.47673    1.13353    8.93487
 -0.480372   1.12683    8.94004
  1.51386    1.1246     8.94155
  3.51035    1.12419    8.93422
 -1.47581    2.85802    8.93404
  0.517145   2.85329    8.93884
  2.51268    2.85055    8.94026